<a href="https://colab.research.google.com/github/german1728/Chimera/blob/master/Tutorial1_Statistical_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 1 - Machine Learning
# Regresión Lineal con MSE

**Content creators**: Pierre-Étienne Fiquet, Anqi Wu, Alex Hyafil with help from Byron Galbraith

**Content reviewers**: Lina Teichmann, Saeed Salehi, Patrick Mineault,  Ella Batty, Michael Waskom

**Content traduction**: Israel Chaparro

Source: Neuromatch Academy




___
#Objetivos del Tutorial

Este es el Tutorial 1 de una serie sobre cómo ajustar modelos a datos. Comenzamos con regresión lineal simple, usando optimización de mínimos cuadrados (Tutorial 1) y Estimación de máxima verosimilitud (Tutorial 2). Usaremos bootstrapping para construir intervalos de confianza alrededor de los parámetros del modelo lineal inferido (Tutorial 3). Terminaremos nuestra exploración de modelos de regresión generalizando a la regresión lineal múltiple y la regresión polinomial (Tutorial 4). Terminamos aprendiendo a elegir entre estos distintos modelos. Analizamos la compensación de sesgo-varianza (Tutorial 5) y la Validación cruzada para la selección del modelo (Tutorial 6).

En este tutorial, aprenderemos cómo ajustar modelos lineales simples a los datos.
- Aprenda a calcular el error cuadrático medio (MSE)
- Explore cómo los parámetros del modelo (pendiente) influyen en el MSE
- Aprenda a encontrar el parámetro de modelo óptimo mediante la optimización de mínimos cuadrados

---

---
# Configuraciones

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#@title Configuraciones de Figuras
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

In [ ]:
#@title Funciones de ayuda

def plot_observed_vs_predicted(x, y, y_hat, theta_hat):
  """ Plot observed vs predicted data

  Args:
      x (ndarray): observed x values
  y (ndarray): observed y values
  y_hat (ndarray): predicted y values

  """
  fig, ax = plt.subplots()
  ax.scatter(x, y, label='Observed')  # our data scatter plot
  ax.plot(x, y_hat, color='r', label='Fit')  # our estimated model
  # plot residuals
  ymin = np.minimum(y, y_hat)
  ymax = np.maximum(y, y_hat)
  ax.vlines(x, ymin, ymax, 'g', alpha=0.5, label='Residuals')
  ax.set(
      title=fr"$\hat{{\theta}}$ = {theta_hat:0.2f}, MSE = {mse(x, y, theta_hat):.2f}",
      xlabel='x',
      ylabel='y'
  )
  ax.legend()

---
# Sección 1: Error cuadrático medio (MSE)

**La regresión lineal de mínimos cuadrados** es un procedimiento de optimización antiguo pero de oro que vamos a utilizar para el ajuste de datos. Los problemas de optimización de mínimos cuadrados (LS) son aquellos en los que la función objetivo es una función cuadrática de los parámetros que se optimizan.

Suponga que tiene un conjunto de medidas, $ y_ {n} $ (la variable "dependiente") obtenida para diferentes valores de entrada, $ x_ {n} $ (la variable "independiente" o "explicativa"). Supongamos que creemos que las medidas son proporcionales a los valores de entrada, pero están dañadas por algunos errores de medida (aleatorios), $ \epsilon_ {n} $, es decir:

$$y_{n}= \theta x_{n}+\epsilon_{n}$$

para algún parámetro de pendiente desconocido $ \ theta. $ El problema de regresión de mínimos cuadrados utiliza ** error cuadrático medio (MSE) ** como su función objetivo, su objetivo es encontrar el valor del parámetro $ \theta $ minimizando el promedio de cuadrados errores:

\begin{align}
\min _{\theta} \frac{1}{N}\sum_{n=1}^{N}\left(y_{n}-\theta x_{n}\right)^{2}
\end{align}

Ahora exploraremos cómo se usa MSE para ajustar un modelo de regresión lineal a los datos. Con fines ilustrativos, crearemos un conjunto de datos sintéticos simple donde conocemos el verdadero modelo subyacente. Esto nos permitirá ver cómo se comparan nuestros esfuerzos de estimación para descubrir el modelo real (aunque en la práctica rara vez tenemos este lujo).

Primero, generaremos algunas muestras ruidosas $ x $ de [0, 10) a lo largo de la línea $ y = 1.2x $ como nuestro conjunto de datos al que deseamos ajustar un modelo.

In [ ]:
# @title

# @markdown Ejecute esta celda para generar algunos datos simulados

# setting a fixed seed to our random number generator ensures we will always
# get the same psuedorandom number sequence
np.random.seed(121)

# Let's set some parameters
theta = 1.2
n_samples = 30

# Draw x and then calculate y
x = 10 * np.random.rand(n_samples)  # sample from a uniform distribution over [0,10)
noise = np.random.randn(n_samples)  # sample from a standard normal distribution
y = theta * x + noise

# Plot the results
fig, ax = plt.subplots()
ax.scatter(x, y)  # produces a scatter plot
ax.set(xlabel='x', ylabel='y');

Ahora que tenemos nuestro conjunto de datos adecuadamente ruidoso, podemos comenzar a intentar estimar el modelo subyacente que lo produjo. Usamos MSE para evaluar qué tan exitosa es una estimación de pendiente particular $ \hat {\theta} $ para explicar los datos, cuanto más cerca de 0 esté el MSE, mejor se ajusta nuestra estimación a los datos.

## Ejercicio 1: Computar el MSE

En este ejercicio, implementará un método para calcular el error cuadrático medio para un conjunto de entradas $ x $, medidas $ y $ y estimación de pendiente $ \hat{\theta} $. Luego calcularemos e imprimiremos el error cuadrático medio para 3 opciones diferentes de theta

In [ ]:
def mse(x, y, theta_hat):
  """Compute the mean squared error

  Args:
    x (ndarray): Un array de tamaño (samples,) que contiene los valores de entrada.
    y (ndarray): Un array de tamaño (samples,) que contiene los valores de medición correspondientes a las entradas.
    theta_hat (float): Un estimado del parámetro de pendiente

  Returns:
    float: El error cuadrático medio de los datos con el parámetro estimado.
  """

  # Computar el y estimado
  y_hat = ...

  # Computar el error cuadrático medio
  mse = ...

  return mse


# Descomente a continuación para probar su función
theta_hats = [0.75, 1.0, 1.5]
# for theta_hat in theta_hats:
#   print(f"theta_hat de {theta_hat} tiene un MSE de {mse(x, y, theta_hat):.2f}")

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W1D3_ModelFitting/solutions/W1D3_Tutorial1_Solution_12a57de0.py)



El resultado debe ser:

theta_hat of 0.75 has an MSE of 9.08\
theta_hat of 1.0 has an MSE of 3.0\
theta_hat of 1.5 has an MSE of 4.52





Vemos que $ \hat{\theta} = 1.0 $ es nuestra mejor estimación de las tres que probamos. Sin embargo, mirar solo los números sin procesar no siempre es satisfactorio, así que visualicemos cómo se ve nuestro modelo estimado sobre los datos.

In [ ]:
#@title

#@markdown Ejecute esta celda para visualizar modelos estimados

fig, axes = plt.subplots(ncols=3, figsize=(18, 4))
for theta_hat, ax in zip(theta_hats, axes):

  # True data
  ax.scatter(x, y, label='Observed')  # our data scatter plot

  # Compute and plot predictions
  y_hat = theta_hat * x
  ax.plot(x, y_hat, color='r', label='Fit')  # our estimated model

  ax.set(
      title= fr'$\hat{{\theta}}$= {theta_hat}, MSE = {mse(x, y, theta_hat):.2f}',
      xlabel='x',
      ylabel='y'
  );

axes[0].legend()

## Demo Interactivo: Explorador de MSE

Usando un widget interactivo, podemos ver fácilmente cómo cambiar nuestra estimación de pendiente cambia el ajuste de nuestro modelo. Mostramos los **residuales**, las diferencias entre los datos observados y los predichos, como segmentos de línea entre el punto de datos (respuesta observada) y la respuesta predicha correspondiente en la línea de ajuste del modelo.

In [ ]:
#@title

#@markdown ¡Asegúrese de ejecutar esta celda para habilitar el widget!

@widgets.interact(theta_hat=widgets.FloatSlider(1.0, min=0.0, max=2.0))
def plot_data_estimate(theta_hat):
  y_hat = theta_hat * x
  plot_observed_vs_predicted(x, y, y_hat, theta_hat)

NameError: ignored

Si bien explorar visualmente varias estimaciones puede ser instructivo, no es la más eficiente para encontrar la mejor estimación que se ajuste a nuestros datos. Otra técnica que podemos usar es elegir un rango razonable de valores de parámetros y calcular el MSE en varios valores en ese intervalo. Esto nos permite graficar el error contra el valor del parámetro (esto también se llama **panorama de errores**, especialmente cuando tratamos con más de un parámetro). Podemos seleccionar el $ \hat{\theta} $ ($ \hat{\theta}_{MSE} $) final como el que resulte en el error más bajo.

In [ ]:
# @title

# @markdown Ejecute esta celda para recorrer theta_hats, calcular MSE y trazar resultados

# Loop over different thetas, compute MSE for each
theta_hat_grid = np.linspace(-2.0, 4.0)
errors = np.zeros(len(theta_hat_grid))
for i, theta_hat in enumerate(theta_hat_grid):
  errors[i] = mse(x, y, theta_hat)

# Find theta that results in lowest error
best_error = np.min(errors)
theta_hat = theta_hat_grid[np.argmin(errors)]


# Plot results
fig, ax = plt.subplots()
ax.plot(theta_hat_grid, errors, '-o', label='MSE', c='C1')
ax.axvline(theta, color='g', ls='--', label=r"$\theta_{True}$")
ax.axvline(theta_hat, color='r', ls='-', label=r"$\hat{{\theta}}_{MSE}$")
ax.set(
  title=fr"Best fit: $\hat{{\theta}}$ = {theta_hat:.2f}, MSE = {best_error:.2f}",
  xlabel=r"$\hat{{\theta}}$",
  ylabel='MSE')
ax.legend();

Podemos ver que nuestro mejor ajuste es $ \hat{\theta} = 1,18 $ con un MSE de 1,45. ¡Esto está bastante cerca del valor verdadero original $ \ theta = 1.2 $!

---
# Sección 2: Optimización de mínimos cuadrados


Si bien el enfoque detallado anteriormente (calcular MSE en varios valores de $ \hat\theta $) nos llevó rápidamente a una buena estimación, aún se basaba en evaluar el valor de MSE en una cuadrícula de valores especificados a mano. Si no elegimos un buen rango para empezar, o con suficiente granularidad, podríamos perder el mejor estimador posible. Vayamos un paso más allá, y en lugar de encontrar el MSE mínimo a partir de un conjunto de estimaciones candidatas, resolvamos analíticamente.

Podemos hacer esto minimizando la función de costo. El error cuadrático medio es una función objetivo convexa, por lo tanto, podemos calcular su mínimo mediante el cálculo. Después de calcular el mínimo, encontramos que:

\begin{align}
\hat\theta = \frac{\vec{x}^\top \vec{y}}{\vec{x}^\top \vec{x}}
\end{align}

### Ejercicio 2: Resuelva para el estimador óptimo

En este ejercicio, escribirás una función que encuentre el valor óptimo de $ \hat{\theta} $ usando el enfoque de optimización de mínimos cuadrados (la ecuación anterior) para resolver la minimización de MSE. Debe tomar los argumentos $ x $ y $ y $ y devolver la solución $ \hat{\theta} $.

Luego usaremos su función para calcular $ \hat {\theta} $ y graficar la predicción resultante sobre los datos.

In [1]:
def solve_normal_eqn(x, y):
  """Resolver las ecuaciones normales para producir el valor de theta_hat que minimiza MSE.

    Args:
    x (ndarray): Un array de tamaño (samples,) que contiene los valores de entrada.
    y (ndarray): Un array de tamaño (samples,) que contiene los valores de medición correspondientes a las entradas.

  Returns:
    float: el valor de theta_hat arribado de minizar el MSE
  """

  # Computar theta_hat analíticamente
  theta_hat = ...

  return theta_hat


# Descomente a continuación para probar su función
# theta_hat = solve_normal_eqn(x, y)
# y_hat = theta_hat * x
# plot_observed_vs_predicted(x, y, y_hat, theta_hat)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W1D3_ModelFitting/solutions/W1D3_Tutorial1_Solution_7a89ba24.py)

*Example output:*

<img alt='Solution hint' align='left' width=558 height=414 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/W1D3_ModelFitting/static/W1D3_Tutorial1_Solution_7a89ba24_0.png>



Vemos que la solución analítica produce un resultado incluso mejor que nuestra búsqueda de cuadrícula anterior, ¡produciendo $ \hat{\theta} = 1.21 $ con MSE = 1.43!

---
# Resumen

- La regresión lineal de mínimos cuadrados es un procedimiento de optimización que se puede utilizar para el ajuste de datos:
     - Tarea: predice un valor para $ y $ dado $ x $
     - Medida de rendimiento: $ \textrm {MSE} $
     - Procedimiento: minimizar $ \textrm {MSE} $ resolviendo las ecuaciones normales
- **Punto clave**: ajustamos el modelo definiendo una *función objetivo* y minimizándola.
- **Nota**: En este caso, existe una solución *analítica* para el problema de minimización y, en la práctica, esta solución se puede calcular usando *álgebra lineal*. Esto es *extremadamente* poderoso y forma la base de gran parte del cálculo numérico en todas las ciencias.

---
# Appendix

## Derivación de optimización de mínimos cuadrados

Aquí describiremos la derivación de la solución de mínimos cuadrados.

Primero establecemos la derivada de la expresión de error con respecto a $ \theta $ igual a cero,

\begin{align}
\frac{d}{d\theta}\frac{1}{N}\sum_{i=1}^N(y_i - \theta x_i)^2 = 0 \\
\frac{1}{N}\sum_{i=1}^N-2x_i(y_i - \theta x_i) = 0
\end{align}

donde usamos la regla de la cadena. Ahora, despejando $ \theta $, obtenemos un valor óptimo de:

\begin{align}
\hat\theta = \frac{\sum_{i=1}^N x_i y_i}{\sum_{i=1}^N x_i^2}
\end{align}

Que podemos escribir en notación vectorial como:

\begin{align}
\hat\theta = \frac{\vec{x}^\top \vec{y}}{\vec{x}^\top \vec{x}}
\end{align}